In [12]:
import holoviews as hv
import hvplot.xarray  # noqa: F401
import numpy as np
import xarray as xr
from holoviews import opts
import json
import s3fs
from odc.stac import configure_rio, load
import pystac
import rioxarray

from utils import get_rgb_dataset, hv_stream_to_rio_geometries, get_earthdata_token

In [13]:
import odc.stac
if odc.stac.__version__ != "0.3.6":
    raise Exception(f"You need to use odc.stac version 0.3.6 or greater. You have {odc.stac.__version__}")

In [15]:
%%time
s3_uri = "s3://files.auspatious.com/hsi_example/TD1_004930_20230205_L2A_20230224_03001065_COG.stac-item.json"

# Open S3 object as a file using s3fs
s3 = s3fs.S3FileSystem(anon=True)
with s3.open(s3_uri, "rt") as f:
    stac_dict = json.load(f)
item = pystac.read_dict(stac_dict)

# Optionally select a subset, so it doesn't take a long time
eo_bands_subset = item.assets["reflectance"].extra_fields["eo:bands"]

# Load the data, telling rasterio to not sign requests
configure_rio(
    cloud_defaults=True,
    aws={"aws_unsigned": True},
    AWS_S3_ENDPOINT="s3.ap-southeast-2.amazonaws.com",
)
ds = load(
    [item],
    measurements=[i["name"] for i in eo_bands_subset],
    chunks={"bands": 1, "longitude": 1200, "latitude": 1200}
)

# No need for time
ds = ds.squeeze("time")

# Stack up the bands, so we have a multi-dimensional raster instead
ds_stacked = ds.to_array("bands")

# Replace the original ds object with a nice indexed one
bands = list([float(i["description"]) for i in eo_bands_subset])
bands.sort()  # This is pretty dangerous... let's assume the .tif has bands in the right order!
ds = ds_stacked.assign_coords(bands=bands).to_dataset(name="reflectance")

# mask 0 as nan
ds = ds.where(ds != 0)

# Load all the data. Should take less than 2 minutes
ds = ds.compute()

ds


CPU times: user 22min 42s, sys: 3min 55s, total: 26min 37s
Wall time: 1min 51s


<xarray.Dataset>
Dimensions:      (bands: 65, latitude: 3318, longitude: 2229)
Coordinates:
  * latitude     (latitude) float64 -34.91 -34.91 -34.91 ... -35.81 -35.81
  * longitude    (longitude) float64 143.4 143.4 143.4 ... 144.0 144.0 144.0
    spatial_ref  int32 4326
    time         datetime64[ns] 2023-02-05
  * bands        (bands) float64 486.0 490.0 493.0 497.0 ... 893.5 895.0 896.0
Data variables:
    reflectance  (bands, latitude, longitude) float64 nan nan nan ... nan nan

In [ ]:
# Select wavelengths to be displayed
r = 680
g = 550
b = 465

# Select brightness - range between 0-1, higher values 'brighten' the whole scene
brightness = 0.2

## End configuration area ##

# Get data structured for rendering
ds_rgb = get_rgb_dataset(ds, [r, g, b], [0.4, 0.45, 0.45])

In [ ]:
ds_rgb.hvplot.rgb(
    x="longitude", y="latitude", bands="bands", aspect="equal", frame_width=600
)

In [ ]:
# Limit the number of drawn polygons
POLY_LIMIT = 5

color_cycle = hv.Cycle('Category10')
colors = [color_cycle[i] for i in range(5)]

# RGB image/map
map = ds_rgb.hvplot.rgb(
    x="longitude", y="latitude", bands="bands", aspect="equal", frame_width=600
)

# Set up a holoviews points array to enable plotting of the clicked points
xmid = ds.longitude.values[int(len(ds.longitude) / 2)]
ymid = ds.latitude.values[int(len(ds.latitude) / 2)]
polygons = hv.Polygons(
    [],
    kdims=["xs", "ys"],
)

polygons_stream = hv.streams.PolyDraw(
    # data=polygons.columns(),
    source=polygons,
    num_objects=POLY_LIMIT,
    styles={'fill_color': color_cycle.values[0:POLY_LIMIT]}
)

# Plot the Map and Dynamic Map side by side
(map * polygons)

In [ ]:
# Build a spectral plot for each of the drawn polygons.
# This takes a while.

plots = []

geometries = hv_stream_to_rio_geometries(polygons_stream.data)
export = []

for i, geometry in enumerate(geometries):
    data = ds.reflectance.rio.clip(geometry, drop=False)
    hv_data = hv.Dataset(data, kdims=["bands", "latitude", "longitude"], vdims=["reflectance"])
    agg = hv_data.aggregate("bands", np.nanmean, spreadfn=np.nanstd)
    
    data = [i, json.dumps(geometry)] + list(agg.data.reflectance.values) + list(agg.data.reflectance_nanstd.values)
    export.append(data)                                                    

    plots.append(
        (hv.Spread(agg) * hv.Curve(agg, label=f"{i}"))
    )

hv.Overlay(plots).opts(
    opts.Spread(color=color_cycle),
    opts.Curve(color=color_cycle),
    opts.Overlay(show_title=False, frame_width=600, show_legend=False)
)

In [ ]:
import csv 

data = polygons_stream.data
wavelengths = ds.wavelengths.values

rows = [["id", "geom"] + [f"{i}_m" for i in wavelengths] + [f"{i}_std" for i in wavelengths]]

rows = rows + export
    
len(rows)

In [ ]:
with open('polygon_data.csv', 'w') as f:
    writer = csv.writer(f, delimiter=";")
    writer.writerows(rows)